In [ ]:
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import time
from scipy import stats

output_folder=r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202_Analysis'

#sandwell coordinate dictionary#
swd={'SW1':(30.1, 8.6), 'SW2':(68.3, 28.5), 'SW3':(29.8, 70.9)}
swn=['SW1', 'SW2', 'SW3']

# bins #
x_bins=np.linspace(4.8, 94.8, 9, endpoint=False)
x_bins = np.insert(x_bins, 0, -5.2, axis=0)
x_bins = np.insert(x_bins, 10, 94.8, axis=0)
x_bins = np.insert(x_bins, 11, 104.8, axis=0)
y_bins=np.linspace(3.6, 93.6, 9, endpoint=False)
y_bins = np.insert(y_bins, 0, -6.4, axis=0)
y_bins = np.insert(y_bins, 10, 93.6, axis=0)
y_bins = np.insert(y_bins, 11, 103.6, axis=0)
y_bins = np.insert(y_bins, 12, 113.6, axis=0)
y_bins = np.insert(y_bins, 13, 123.6, axis=0)


#read in relevant files
N_events_data=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\LR_data\H2202_NAL_events_translate.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
I_events_data=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\LR_data\H2202_IAL_events_translate.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
A_events_data=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\LR_data\H2202_AAL_events_translate.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
P_events_data=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\LR_data\H2202_PAL_events_translate.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

#read in relevant place cell data 
#N01_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_N01_PRE_MI.csv')
#N01_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_N01_PRE_SD.csv')
#N02_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_N02_PRE_MI.csv')
#N02_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_N02_PRE_SD.csv')
#I01_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_I01_PRE_MI.csv')
#I01_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_I01_PRE_SD.csv')
#I02_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_I02_PRE_MI.csv')
#I02_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_I02_PRE_SD.csv')
A01_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_A01_PRE_MI.csv')
A01_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_A01_PRE_SD.csv')
A02_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_A02_PRE_MI.csv')
A02_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_A02_PRE_SD.csv')
P01_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_P01_PRE_MI.csv')
P01_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_P01_PRE_SD.csv')
P02_MI=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_P02_PRE_MI.csv')
P02_SD=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\Mutual Information\H2202_P02_PRE_SD.csv')

In [ ]:
## creating dictionary ##
Animal='H2202'
sessions=['N01', 'N02', 'I01', 'I02', 'A01', 'A02', 'P01', 'P02']
pcsessions=['A01', 'A02', 'P01', 'P02']
stages=['PRE', 'SAM', 'CHO']
stagesp=['PRE', 'SAM', 'CHO', 'PRO']

xydict={}
for i in sessions:
    xydict[i]={}
z=list(xydict.keys())[:6]
for l in z:
    for j in stages:
        xydict[l][j]={}
p=list(xydict.keys())[6:]
for r in p:
    for i in stagesp:
        xydict[r][i]={}

pc_dict={'A01_MI':A01_MI, 'A01_SD':A01_SD, 'A02_MI':A02_MI, 'A02_SD':A02_SD,'P01_MI':P01_MI, 'P01_SD':P01_SD,  'P02_MI':P02_MI, 'P02_SD':P02_SD }

In [ ]:
## establishing corresponsdence ##
# NB: may need to iterate removal of objects with length less than 2 #
corr_table=pd.read_csv(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\LR_data\H2202_123_APx_123_COR.csv')
gci=corr_table['global_cell_index'].unique()
corr_vals=[]
for i in gci:
    subbed=corr_table[corr_table['global_cell_index']==i]
    corr_vals.append(subbed['local_cell_index'].unique().tolist())
for i in corr_vals:
    if len(i)<2:   # may need to be repeated to remove all non corresponding ids
        corr_vals.remove(i)
corr_vals   

In [ ]:
### storing id of rewarded sandwell ###

for s in sessionss:
    for p in list(xydict[s].keys()):
        swn=['SW1', 'SW2', 'SW3']
        if s.startswith('A'):
            events_data=A_events_data
            f=' C'
        else:
            events_data=P_events_data
            f=' C'
        ss_data=events_data[events_data['Session']==s]
        ss_data=ss_data[ss_data['stage']==p]
        ss_data=ss_data[ss_data['Movement status']=='moving']
        cell_events=ss_data[[col for col in ss_data if col.startswith(f)]]
        rewarded_sandwell=ss_data['well'].unique().tolist()[0]
        xydict[s][p]['RW']=rewarded_sandwell

In [ ]:
#### EUCLID DIST OF PLACE FIELD CENTRE TO RW ####
# may need this if running pc analysis before reward cells # (HAVE SORTED INTO CELL TYPE - will need ED_RW stored prior pfc distributions)

#sessionss=['A01', 'A02', 'P01', 'P02']
#for t in sessionss:
#    for e in list(xydict[t].keys()):
#        RW_ED=[]
 #       for i in xydict[t][e]['pfc']:
  #          x_ed=i[0]-swd[xydict[t][e]['RW']][0]
   #         y_ed=i[1]-swd[xydict[t][e]['RW']][1]
    #        RW_ED.append(math.sqrt(x_ed**2+y_ed**2))
     #   xydict[t][e]['ED_RW']=RW_ED

In [ ]:
### PLACE CELL ANALYSIS ###

In [ ]:
### correspondence of place cells ###
# A01/A02 #
A_pcs=[]
for i in xydict['A01']['PRE']['pc_id']:
    if i in xydict['A02']['PRE']['pc_id']:
        A_pcs.append(i)
# P01/P02 #
P_pcs=[]
for i in xydict['P01']['PRE']['pc_id']:
    if i in xydict['P02']['PRE']['pc_id']:
        P_pcs.append(i)
        
# finding associated advanced index for probe trial place cells #
P01_adval=[]
for i in xydict['P01']['PRE']['pc_id']:
    for j in corr_vals:
        if j[1]==i:
            P01_adval.append(j[0])
P02_adval=[]
for i in xydict['P02']['PRE']['pc_id']:
    for j in corr_vals:
        if j[1]==i:
            P02_adval.append(j[0])

# finding associated probe index for advanced trial place cells #
A01_proval=[]
for i in xydict['A01']['PRE']['pc_id']:
    for j in corr_vals:
        if j[0]==i:
            A01_proval.append(j[1])
A02_proval=[]
for i in xydict['A02']['PRE']['pc_id']:
    for j in corr_vals:
        if j[0]==i:
            A02_proval.append(j[1])

In [ ]:
## CELLS MUTUAL TO TWO SESSIONS - PERMUTED BETWEEN SESSIONS ##

# appears in A01 / P01 #
#in probe id#
A01_P01_pcs=[]
for i in A01_proval:
    if i in xydict['P01']['PRE']['pc_id']:
        A01_P01_pcs.append(i)
#reverse check
#in adv id#
P01_A01_pcs=[]
for i in P01_adval:
    if i in xydict['A01']['PRE']['pc_id']:
        P01_A01_pcs.append(i)

        
# appears in A01 / P02 #
#in probe id#
A01_P02_pcs=[]
for i in A01_proval:
    if i in xydict['P02']['PRE']['pc_id']:
        A01_P02_pcs.append(i)
#reverse check
#in adv id#
P02_A01_pcs=[]
for i in P02_adval:
    if i in xydict['A01']['PRE']['pc_id']:
        P02_A01_pcs.append(i)

# appears in A02 / P01 #
#in probe id#
A02_P01_pcs=[]
for i in A02_proval:
    if i in xydict['P01']['PRE']['pc_id']:
        A02_P01_pcs.append(i)
#reverse check
#in adv id#
P01_A02_pcs=[]
for i in P01_adval:
    if i in xydict['A02']['PRE']['pc_id']:
        P01_A02_pcs.append(i)

# appears in A02 / P02 #
#in probe id#
A02_P02_pcs=[]
for i in A02_proval:
    if i in xydict['P02']['PRE']['pc_id']:
        A02_P02_pcs.append(i)
#reverse check
#in adv id#
P02_A02_pcs=[]
for i in P02_adval:
    if i in xydict['A02']['PRE']['pc_id']:
        P02_A02_pcs.append(i)

In [ ]:
## CHECKING FOR PLACE CELLS MUTUAL TO THREE SESSIONS ##
# in A01/A02/P01 #
A01_A02_P01_pcs=[]
P01_A01_A02_pcs=[]
for i in A_pcs:
    for j in corr_vals:
        if j[0]==i:
            if j[1] in xydict['P01']['PRE']['pc_id']:
                A01_A02_P01_pcs.append(i)
                P01_A01_A02_pcs.append(j[1])

# in A01/A02/P02 #
A01_A02_P02_pcs=[]
P02_A01_A02_pcs=[]
for i in A_pcs:
    for j in corr_vals:
        if j[0]==i:
            if j[1] in xydict['P02']['PRE']['pc_id']:
                A01_A02_P02_pcs.append(i)
                P02_A01_A02_pcs.append(j[1])

# in A01/P01/P02 #
A01_P01_P02_pcs=[]
P01_P02_A01_pcs=[]
for i in P_pcs:
    for j in corr_vals:
        if j[1]==i:
            if j[0] in xydict['A01']['PRE']['pc_id']:
                A01_P01_P02_pcs.append(j[0])
                P01_P02_A01_pcs.append(i)
                
# in A02/P01/P02 #
A02_P01_P02_pcs=[]
P01_P02_A02_pcs=[]
for i in P_pcs:
    for j in corr_vals:
        if j[1]==i:
            if j[0] in xydict['A02']['PRE']['pc_id']:
                A02_P01_P02_pcs.append(j[0])
                P01_P02_A02_pcs.append(i)

In [ ]:
## MUTUAL TO FOUR SESSIONS ##
# 4 sessions - adv index #
all_four_pc=[]
for i in A01_A02_P02_pcs:
    for j in corr_vals:
        if j[0]==i:
            if j[1] in xydict['P01']['PRE']['pc_id']:
                all_four_pc.append(i)

In [ ]:
### FINDING PLACE FIELD CENTRE FOR ALL CELLS ###

#Place field centres all cells#
sessionss=['A01', 'A02', 'P01', 'P02']
for t in sessionss:
    for e in list(xydict[t].keys()):
        if t.startswith('A'):
            events_data=A_events_data
            f=' C'
        if t.startswith('P'):
            events_data=P_events_data
            f=' C'
        if t.startswith('N'):
            events_data=N_events_data
            f=' C'
        if t.startswith('I'):
            events_data=I_events_data
            f='C'
            
        ss_data=events_data[events_data['Session']==t]
        ss_data=ss_data[ss_data['stage']==e]
        ss_data=ss_data[ss_data['Movement status']=='moving']
        cell_events=ss_data[[col for col in ss_data if col.startswith(f)]]
        cell_events=cell_events.astype(float)
        

        normed_cells=[]
        for j in cell_events:
            cellist=cell_events[j].tolist()
            mino=min(cellist)
            maxo=max(cellist)
            sing=[]
            if maxo!=0:
                for i in cellist:
                    sing.append((i-mino)/(maxo-mino))
                normed_cells.append(sing)
            else:
                normed_cells.append(cellist)
        cell_events=pd.DataFrame(normed_cells)
        cell_events=cell_events.transpose()

        cell_events=cell_events.astype(float)




        #cell_events=cell_events.reset_index(drop=True)
        #binarise data
        onevals=cell_events>=0.3
        zerovals=cell_events<0.3
        cell_events[onevals]=1
        cell_events[zerovals]=0
   
        x=ss_data['x']
        y=ss_data['y']
        #formatting
        newx=[]
        newy=[]
        for i in x:
            newx.append(float(i))
        for j in y:
            newy.append(float(j))

        x_b=x.tolist()
        y_b=y.tolist()
        for i in x_b:
            i = float(i)
        for i in y_b:
            i = float(i)

        occupancy_map_x=[]
        occupancy_map_y=[]
        for x_c, y_c in zip(newx,newy):
            for val in range(len(y_bins)):
                if val<=len(y_bins)-1:
                    if y_bins[val]<=y_c<=y_bins[val+1]:
                        for valx in range(len(x_bins)):
                            if x_bins[valx]<=x_c<=x_bins[valx+1]:
                                occupancy_map_x.append(valx)
                                occupancy_map_y.append(val)
        OM=list(zip(occupancy_map_x, occupancy_map_y))
        event_bins=[]
        for j in cell_events:
            cebins=[]
            i=0
            while i<len(cell_events[j]):
                if cell_events[j][i]==0:
                    cebins.append(99999)
                    i+=1
                else:
                    loc=OM[i]
                    cebins.append(loc)
                    i+=1
            event_bins.append(cebins)
        all_locs=[] #list of lists - each sublist represents the cells maximal bins 
        p=0
        while p<len(event_bins): #going through cell by cell
            uniq=[]  #finding all unique firing locations
            q=0
            while q<len(event_bins[p]):
                if event_bins[p][q] not in uniq:
                    if event_bins[p][q]!=99999:  #appending unique event locations
                        uniq.append(event_bins[p][q])
                        q+=1
                    else:
                        q+=1
                else:
                    q+=1
            freq=[]
            for r in uniq:
                freq.append(event_bins[p].count(r)) #counting frequency of each event bin

            if not freq:
                all_locs.append([(7,7),(5,5),(3,5,6), (6,7,7)])
                p+=1
            else:
                m=max(freq)
                pos=[i for i, j in enumerate(freq) if j == m]
                cell_max_locs=[]
                z=0
                while z<len(pos):
                    cell_max_locs.append(uniq[pos[z]])
                    z+=1
                all_locs.append(cell_max_locs)
                p+=1
        
        
        j=0
        max_centres=[]
        while j<len(all_locs):
            if len(all_locs[j])==1:
                i=0
                max_x=[]
                max_y=[]
                while i<len(cell_events):
                    if cell_events[j][i]!=0:
                        for p, q in all_locs[j]:
                            if x_bins[p]<=float(x_b[i])<=x_bins[p+1]:
                                if y_bins[q]<=float(y_b[i])<=y_bins[q+1]:
                                    max_x.append(float(x_b[i]))
                                    max_y.append(float(y_b[i]))
                                    i+=1
                                else:
                                    i+=1
                            else:
                                i+=1
                    else:
                        i+=1
                max_centres.append((np.mean(max_x), np.mean(max_y)))
                j+=1
            else:    
                if len(all_locs[j])==2:
                    if all_locs[j][0][0]==all_locs[j][1][0] or all_locs[j][0][1]==all_locs[j][1][1]:
                        max_x=[]
                        max_y=[]
                        i=0
                        while i<len(cell_events):
                            if cell_events[j][i]!=0:
                                p=all_locs[j][0][0]
                                q=all_locs[j][0][1]
                                r=all_locs[j][1][0]
                                s=all_locs[j][1][1]
                                if x_bins[p]<=float(x_b[i])<=x_bins[p+1]:
                                    if y_bins[q]<=float(y_b[i])<=y_bins[q+1]:
                                        max_x.append(float(x_b[i]))
                                        max_y.append(float(y_b[i]))
                                        i+=1
                                    else:
                                        i+=1
                                if x_bins[r]<=float(x_b[i])<=x_bins[r+1]:
                                    if y_bins[s]<=float(y_b[i])<=y_bins[s+1]:
                                        max_x.append(float(x_b[i]))
                                        max_y.append(float(y_b[i]))
                                        i+=1
                                    else:
                                        i+=1
                                else:
                                    i+=1
                            else:
                                i+=1
                        max_centres.append((np.mean(max_x), np.mean(max_y)))
                        j+=1
                    else: 
                        max_centres.append((np.nan, np.nan))
                        j+=1
                        
                else:
                    max_centres.append((np.nan, np.nan))
                    j+=1
        
        place_field_centres=max_centres
        xydict[t][e]['pfc']=place_field_centres

In [ ]:
### REMAPPING RATE OF PLACE CELLS (FIRST DONE SESSION TO SESSION, THEN AVERAGE) ###

#A01_A02#
A_pc_rr=[]
for i in A_pcs:
    init=xydict['A01']['PRE']['pfc'][i]
    fin=xydict['A02']['PRE']['pfc'][i]
    x=init[0]-fin[0]
    y=init[1]-fin[1]
    if x<0 and y>=0 or x>=0 and y<0:
        displacement=(-1)*math.sqrt(x**2+y**2)
    if x>=0 and y>=0 or x<0 and y<0:
        displacement=math.sqrt(x**2+y**2)
    A_pc_rr.append(displacement)

#vals=all_remaps
import matplotlib.mlab as mlab

A_pc_rr_n0=[]
for i in A_pc_rr:
    if i>0 or i<0 or i==0:
        A_pc_rr_n0.append(i)

plt.clf()

(mu, sigma)=stats.norm.fit(A_pc_rr_n0)
n, bins, patches = plt.hist(A_pc_rr_n0, 40, density=True, facecolor='lightblue', alpha=0.75, align='left', edgecolor='royalblue' , linewidth=1.2)
y = stats.norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.7)
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.title('H2202 Place Cell remapping rate A01/A02', pad=25)
plt.plot([], [], ' ', label=" μ = -0.5604057759249593, σ = 3.598680566480656")
plt.xlabel('Remapping distance / cm')
plt.ylabel('Proportion of cells / %')
lgd=plt.legend(loc=1, prop={'size': 10}, bbox_to_anchor=(2, 1.05))
new=[0.0, 5.0, 10.0, 15.0, 20.0, 25.0,30.0, 35.0]
#plt.yticks(ticks=locs, labels=new)
#plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\%s_A01_A02_remapping_rate.png' %(Animal), bbox_extra_artists=(lgd,), bbox_inches='tight')

#A02_P01_remapping_rate#
AP_pc_rr=[]
i=0
while i<len(A02_P01_pcs):
    init=xydict['A02']['PRE']['pfc'][P01_A02_pcs[i]]
    fin=xydict['P02']['PRE']['pfc'][A02_P01_pcs[i]]
    x=init[0]-fin[0]
    y=init[1]-fin[1]
    if x<0 and y>=0 or x>=0 and y<0:
        displacement=(-1)*math.sqrt(x**2+y**2)
    if x>=0 and y>=0 or x<0 and y<0:
        displacement=math.sqrt(x**2+y**2)
    AP_pc_rr.append(displacement)
    i+=1

#vals=all_remaps
import matplotlib.mlab as mlab

AP_pc_rr_n0=[]
for i in AP_pc_rr:
    if i>0 or i<0 or i==0:
        AP_pc_rr_n0.append(i)

plt.clf()

(mu, sigma)=stats.norm.fit(AP_pc_rr_n0)
n, bins, patches = plt.hist(AP_pc_rr_n0, 40, density=True, facecolor='lightblue', alpha=0.75, align='left', edgecolor='royalblue' , linewidth=1.2)
y = stats.norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.7)
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.title('H2202 Place Cell remapping rate A02/P01', pad=25)
plt.plot([], [], ' ', label=" μ = 16.107048630958467, σ = 35.43657842021704")
plt.xlabel('Remapping distance / cm')
plt.ylabel('Proportion of cells / %')
#plt.yticks(ticks=locs, labels=lebs)

lgd=plt.legend(loc=1, prop={'size': 10}, bbox_to_anchor=(2, 1.05))
plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\%s_A02_P01_remapping_rate.png' %(Animal), bbox_extra_artists=(lgd,), bbox_inches='tight')
#print(mu, sigma)

#P01_P02_remapping_rate#
P_pc_rr=[]
for i in P_pcs:
    init=xydict['P01']['PRE']['pfc'][i]
    fin=xydict['P02']['PRE']['pfc'][i]
    x=init[0]-fin[0]
    y=init[1]-fin[1]
    if x<0 and y>=0 or x>=0 and y<0:
        displacement=(-1)*math.sqrt(x**2+y**2)
    if x>=0 and y>=0 or x<0 and y<0:
        displacement=math.sqrt(x**2+y**2)
    P_pc_rr.append(displacement)

#vals=all_remaps
import matplotlib.mlab as mlab

P_pc_rr_n0=[]
for i in P_pc_rr:
    if i>0 or i<0 or i==0:
        P_pc_rr_n0.append(i)

plt.clf()

(mu, sigma)=stats.norm.fit(P_pc_rr_n0)
n, bins, patches = plt.hist(P_pc_rr_n0, 40, density=True, facecolor='lightblue', alpha=0.75, align='left', edgecolor='royalblue' , linewidth=1.2)
y = stats.norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.7)
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.title('H2202 Place Cell remapping rate P01/P02', pad=25)
plt.plot([], [], ' ', label=" μ = -0.0705894925109474, σ = 2.4852854720828685")
plt.xlabel('Remapping distance / cm')
plt.ylabel('Proportion of cells / %')
#plt.yticks(ticks=locs, labels=hi)

lgd=plt.legend(loc=1, prop={'size': 10}, bbox_to_anchor=(2, 1.05))
#plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\%s_P01_P02_remapping_rate.png' %(Animal), bbox_extra_artists=(lgd,), bbox_inches='tight')
print(mu, sigma)


## AVERAGE REMAPPING RATE PLACE CELLS ##

APs_pc_rr=[]
for i in A_pc_rr:
    if i>0 or i<0 or i==0:
        APs_pc_rr.append(i)
for i in P_pc_rr:
    if i>0 or i<0 or i==0:
        APs_pc_rr.append(i)
for i in AP_pc_rr:
    if i>0 or i<0 or i==0:
        APs_pc_rr.append(i)
        
(mu, sigma)=stats.norm.fit(APs_pc_rr)
n, bins, patches = plt.hist(APs_pc_rr, 40, density=True, facecolor='lightblue', alpha=0.75, align='left', edgecolor='royalblue' , linewidth=1.2)
y = stats.norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.7)
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.title('H2202 Place Cell remapping rate A01/A02/P01/P02', pad=25)
plt.plot([], [], ' ', label=" μ = -0.5604057759249593, σ = 3.598680566480656")
plt.xlabel('Remapping distance / cm')
plt.ylabel('Proportion of cells / %')
#plt.yticks(ticks=locs, labels=labs)
lgd=plt.legend(loc=1, prop={'size': 10}, bbox_to_anchor=(2, 1.05))
#plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\%s_A01_A02_P01_P02_remapping_rate.png' %(Animal), bbox_extra_artists=(lgd,), bbox_inches='tight')
print(mu, sigma)

In [ ]:
### Place field centres distribution from RW ##

meanss=[]
sdss=[]
all_vals=[]
for t in sessionss:
    for e in ['PRE']:
        plt.clf()
        toplot=[]
        pcids=xydict[t]['PRE']['pc_id']
        for i in pcids:
            toplot.append(xydict[t][e]['ED_RW'][i])
            
        toplot_n0=[]
        for i in toplot:
            if i>0 or i<0 or i==0:
                toplot_n0.append(i)
        all_vals.append(toplot_n0)        
        (mu, sigma)=stats.norm.fit(toplot_n0)
        n, bins, patches = plt.hist(toplot_n0, 40, density=True, facecolor='lightblue', alpha=0.75, align='left', edgecolor='royalblue' , linewidth=1.2)
        y = stats.norm.pdf(bins, mu, sigma)
        l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.7)
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False
        plt.title('H2202 Place Cell Field distribution '+t+' '+e, pad=25)
        locs, labels=plt.yticks()
        new=locs*100
        news=[]
        for i in new:
            news.append(round(i, 1))
            
        plt.yticks(ticks=locs, labels=news)
        meanss.append(mu)
        sdss.append(sigma)
        plt.xlabel('Distance from rewarded sandwell / cm')
        plt.ylabel('Proportion of cells / %')
   #     plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\%s_place_cell_field_dist_%s_%s.png' %(Animal, t, e))
        print(t, e)

In [ ]:
# storing the location of all place cell events #
for s in sessionss:
    for p in ['PRE']:
        if s.startswith('A'):
            events_data=A_events_data
        else:
            events_data=P_events_data
        pc_ids=xydict[s][p]['pc_id']
        ss_data=events_data[events_data['Session']==s]
        ss_data=ss_data[ss_data['stage']==p]
        ss_data=ss_data[ss_data['Movement status']=='moving']
        cell_events=ss_data[[col for col in ss_data if col.startswith(' C')]]
        
        cell_events=cell_events.astype(float)
        
        
        normed_cells=[]
        for j in cell_events:
            cellist=cell_events[j].tolist()
            mino=min(cellist)
            maxo=max(cellist)
            sing=[]
            if maxo!=0:
                for i in cellist:
                    sing.append((i-mino)/(maxo-mino))
                normed_cells.append(sing)
            else:
                normed_cells.append(cellist)
        cell_events=pd.DataFrame(normed_cells)
        cell_events=cell_events.transpose()

        cell_events=cell_events.astype(float)
        
        pc_cell_events=cell_events.copy()
        
        for i in pc_ids:
            pc_cell_events.drop(i, 1)
        
        #cell_events=cell_events.reset_index(drop=True)
        #binarise data
        onevals=pc_cell_events>=0.3
        zerovals=pc_cell_events<0.3
        pc_cell_events[onevals]=1
        pc_cell_events[zerovals]=0
        
        x=ss_data['x']
        y=ss_data['y']
        #formatting
        newx=[]
        newy=[]
        for i in x:
            newx.append(float(i))
        for j in y:
            newy.append(float(j))
        
        xvals=[]
        yvals=[]
        for i in pc_cell_events:
            j=0
            while j<len(pc_cell_events[i]):
                if pc_cell_events[i][j]!=0:
                    xvals.append(float(newx[j]))
                    yvals.append(float(newy[j]))
                    j+=1
                else:
                    j+=1
        xydict[s][p]['x_pc']=xvals
        xydict[s][p]['y_pc']=yvals


        


In [ ]:
#events binned as distance to RWS
meansepc=[]
sdsepc=[]
allvales=[]
for s in sessionss:
    for t in ['PRE']:
        plt.clf()
        ED=[]
        xs= xydict[s][t]['x_pc']
        ys= xydict[s][t]['y_pc']
        rwc=swd[xydict[s][t]['RW']]
        i=0
        while i < len(xs):
            newx=xs[i]-rwc[0]
            newy=ys[i]-rwc[1]
            EDs=math.sqrt(newx**2+newy**2)
            ED.append(EDs)
            i+=1

        vals=ED
        allvales.append(vals)
        (mu, sigma) = stats.norm.fit(vals)
        n, bins, patches = plt.hist(vals, 50, density=True, facecolor='lightblue', align='left', edgecolor='royalblue', alpha=0.75)
       # y = stats.norm.pdf(bins, mu, sigma)
       # l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.4)
        locs, labels=plt.yticks()
        new=locs*100
        news=[]
        for i in new:
            news.append(round(i, 1))
            
        plt.yticks(ticks=locs, labels=news)
        meansepc.append(mu)
        sdsepc.append(sigma)
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False
        plt.ylabel('Proportion of Events / %')
        plt.xlabel('Events distance from rewarded sandwell / Cm')
        plt.title(Animal +' '+ s+' ' + t + ' place cell Event distibution ')
        plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202_Analysis\%s_%s_%s_PC_Events_dist.png' %(Animal, s, t ))

In [ ]:
### REWARD CELL ANALYSIS ###

In [ ]:
#### IDENTIFYING CRCS ####

In [ ]:
### identifying rewarded population ### 

sessionss=['A01', 'A02', 'P01', 'P02']
for t in sessionss:
    for e in list(xydict[t].keys()):
        RW_ED=[]
        for i in xydict[t][e]['pfc']:
            x_ed=i[0]-swd[xydict[t][e]['RW']][0]
            y_ed=i[1]-swd[xydict[t][e]['RW']][1]
            RW_ED.append(math.sqrt(x_ed**2+y_ed**2))
        xydict[t][e]['ED_RW']=RW_ED
        i=0
        feasible_RC=[]
        while i<len(RW_ED):
            if RW_ED[i]<25:
                feasible_RC.append(i)
                i+=1
            else:
                i+=1
        xydict[t][e]['f_RC']=feasible_RC

poss_A01_RC=[]
for i in xydict['A01']['SAM']['f_RC']:
    if i in xydict['A01']['CHO']['f_RC']:
        if i not in xydict['A01']['PRE']['f_RC']:
            poss_A01_RC.append(i)
poss_A02_RC=[]
for i in xydict['A02']['SAM']['f_RC']:
    if i in xydict['A02']['CHO']['f_RC']:
        if i not in xydict['A02']['PRE']['f_RC']:
            poss_A02_RC.append(i)
poss_P01_RC=[]
for i in xydict['P01']['SAM']['f_RC']:
    if i in xydict['P01']['CHO']['f_RC']:
        if i not in xydict['P01']['PRE']['f_RC']:
            poss_P01_RC.append(i)
poss_P02_RC=[]
for i in xydict['P02']['SAM']['f_RC']:
    if i in xydict['P02']['CHO']['f_RC']:
        if i not in xydict['P02']['PRE']['f_RC']:
            poss_P02_RC.append(i)
            
            
## saving corresponding index of A CRCs in P sessions (if appropriate) ##
A01_RC_cp=[]
for i in poss_A01_RC:
    for j in corr_vals:
        if j[0]==i:
            A01_RC_cp.append(j[1])
A02_RC_cp=[]
for i in poss_A02_RC:
    for j in corr_vals:
        if j[0]==i:
            A02_RC_cp.append(j[1])

In [ ]:
### CRC REMAPPING RATE - STAGE/STAGE ###
a01rrs=[]
for i in poss_A01_RC:
    x=xydict['A01']['PRE']['pfc'][i][0]-xydict['A01']['SAM']['pfc'][i][0]
    y=xydict['A01']['PRE']['pfc'][i][1]-xydict['A01']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        a01rrs.append(rr)
a01rr=np.mean(a01rrs)
a01rrrs=[]
for i in poss_A02_RC:
    x=xydict['A01']['CHO']['pfc'][i][0]-xydict['A01']['SAM']['pfc'][i][0]
    y=xydict['A01']['CHO']['pfc'][i][1]-xydict['A01']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        a01rrrs.append(rr)
a01rrr=np.mean(a01rrrs)
a02rrs=[]
for i in poss_A02_RC:
    x=xydict['A02']['PRE']['pfc'][i][0]-xydict['A02']['SAM']['pfc'][i][0]
    y=xydict['A02']['PRE']['pfc'][i][1]-xydict['A02']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        a02rrs.append(rr)
a02rr=np.mean(a02rrs)
a02rrrs=[]
for i in poss_A02_RC:
    x=xydict['A02']['CHO']['pfc'][i][0]-xydict['A02']['SAM']['pfc'][i][0]
    y=xydict['A02']['CHO']['pfc'][i][1]-xydict['A02']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        a02rrrs.append(rr)
a02rrr=np.mean(a02rrrs)
p01rr=[]
p01rrr=[]
p01rrrr=[]
for i in poss_P01_RC:
    x=xydict['P01']['PRE']['pfc'][i][0]-xydict['P01']['SAM']['pfc'][i][0]
    y=xydict['P01']['PRE']['pfc'][i][1]-xydict['P01']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        p01rr.append(rr)
    x=xydict['P01']['PRO']['pfc'][i][0]-xydict['P01']['SAM']['pfc'][i][0]
    y=xydict['P01']['PRO']['pfc'][i][1]-xydict['P01']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        p01rrr.append(rr)
    x=xydict['P01']['PRO']['pfc'][i][0]-xydict['P01']['CHO']['pfc'][i][0]
    y=xydict['P01']['PRO']['pfc'][i][1]-xydict['P01']['CHO']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        p01rrrr.append(rr)

p01rr=np.mean(p01rr)
p01rrr=np.mean(p01rrr)
p01rrrr=np.mean(p01rrrr)        

p02rr=[]
p02rrr=[]
p02rrrr=[]
for i in poss_P02_RC:
    x=xydict['P02']['PRE']['pfc'][i][0]-xydict['P02']['SAM']['pfc'][i][0]
    y=xydict['P02']['PRE']['pfc'][i][1]-xydict['P02']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        p02rr.append(rr)
    x=xydict['P02']['PRO']['pfc'][i][0]-xydict['P02']['SAM']['pfc'][i][0]
    y=xydict['P02']['PRO']['pfc'][i][1]-xydict['P02']['SAM']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        p02rrr.append(rr)
    x=xydict['P02']['PRO']['pfc'][i][0]-xydict['P02']['CHO']['pfc'][i][0]
    y=xydict['P02']['PRO']['pfc'][i][1]-xydict['P02']['CHO']['pfc'][i][1]
    rr=math.sqrt(x**2+y**2)
    if rr>0 or rr<0 or rr==0:
        p02rrrr.append(rr)
p02rr=np.mean(p02rr)
p02rrr=np.mean(p02rrr)
p02rrrr=np.mean(p02rrrr)

fig, ax = plt.subplots(figsize=(15,8))
ax.plot(xlab, [a01rr, a01rrr, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],  marker='x', label='Four cells')
ax.plot(xlab, [np.nan, np.nan, a02rr, a02rrr, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan], marker='x', label='Six cells')
ax.plot(xlab, [np.nan,  np.nan,np.nan,np.nan, p01rr, p01rrr, p01rrrr, np.nan, np.nan, np.nan ], marker='x', label='Four cells')
ax.plot(xlab,[ np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, p02rr, p02rrr, p02rrrr], marker='x', label='Five cells')
lgd=ax.legend(loc=1)
ax.set_title('H2202 Reward cell remapping rate', fontsize=13)
plt.xticks(rotation=70)
ax.set_ylabel('Remapping rate / cm', labelpad=10)
ax.set_xlabel('Session Stage/Stage', labelpad=10)
plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\RewardCellRemapping.png', bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
### GLOBAL ANALYSIS ###

In [ ]:
### overall place field - ALL CELLS - distribution rewarded sandwell all - binned histogram PFC ###
means=[]
sds=[]
rewardedvals=[]
nonrewardedvals=[]
for t in sessionss:
    for e in list(xydict[t].keys()):
        plt.clf()
        toplot=xydict[t][e]['ED_RW']
        toplot_n0=[]
        for i in toplot:
            if i>0 or i<0 or i==0:
                toplot_n0.append(i)
        if e == 'SAM' or e == 'CHO':
            rewardedvals.append(toplot_n0)
        if e == 'PRE' or e == 'PRO':
            nonrewardedvals.append(toplot_n0)
                
        (mu, sigma)=stats.norm.fit(toplot_n0)
        n, bins, patches = plt.hist(toplot_n0, 40, density=True, facecolor='lightblue', alpha=0.75, align='left', edgecolor='royalblue' , linewidth=1.2)
       # y = stats.norm.pdf(bins, mu, sigma)
       # l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.7)
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False
        plt.title('H2202 Place Field distribution '+t+' '+e, pad=25)
        locs, labels=plt.yticks()
        new=locs*100
        news=[]
        for i in new:
            news.append(round(i, 1))
            
        plt.yticks(ticks=locs, labels=news)
        means.append(mu)
        sds.append(sigma)
        plt.xlabel('Distance from rewarded sandwell / cm')
        plt.ylabel('Proportion of cells / %')
        plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\%s_place_field_dist_%s_%s.png' %(Animal, t, e))
        print(t, e)
        

In [ ]:
### storing location of all events in the session ###
for s in sessionss:
    for p in list(xydict[s].keys()):
        swn=['SW1', 'SW2', 'SW3']
        if s.startswith('A'):
            events_data=A_events_data
            f=' C'
        else:
            events_data=P_events_data
            f=' C'
            
        ss_data=events_data[events_data['Session']==s]
        ss_data=ss_data[ss_data['stage']==p]
        ss_data=ss_data[ss_data['Movement status']=='moving']
        cell_events=ss_data[[col for col in ss_data if col.startswith(f)]]
        rewarded_sandwell=ss_data['well'].unique().tolist()[0]
        cell_events=cell_events.astype(float)
        
        
        normed_cells=[]
        for j in cell_events:
            cellist=cell_events[j].tolist()
            mino=min(cellist)
            maxo=max(cellist)
            sing=[]
            if maxo!=0:
                for i in cellist:
                    sing.append((i-mino)/(maxo-mino))
                normed_cells.append(sing)
            else:
                normed_cells.append(cellist)
        cell_events=pd.DataFrame(normed_cells)
        cell_events=cell_events.transpose()

        cell_events=cell_events.astype(float)




        #cell_events=cell_events.reset_index(drop=True)
        #binarise data
        onevals=cell_events>=0.3
        zerovals=cell_events<0.3
        cell_events[onevals]=1
        cell_events[zerovals]=0
        
        x=ss_data['x']
        y=ss_data['y']
        #formatting
        newx=[]
        newy=[]
        for i in x:
            newx.append(float(i))
        for j in y:
            newy.append(float(j))
        
        xvals=[]
        yvals=[]
        for i in cell_events:
            j=0
            while j<len(cell_events[i]):
                if cell_events[i][j]!=0:
                    xvals.append(float(newx[j]))
                    yvals.append(float(newy[j]))
                    j+=1
                else:
                    j+=1
        xydict[s][p]['x']=xvals
        xydict[s][p]['y']=yvals


In [ ]:
#events binned as distance to RWS
meanse=[]
sdse=[]
rewardedvals=[]
nonrewardedvals=[]
for s in sessionss:
    for t in list(xydict[s].keys()):
        plt.clf()
        ED=[]
        xs= xydict[s][t]['x']
        ys= xydict[s][t]['y']
        rwc=swd[xydict[s][t]['RW']]
        i=0
        while i < len(xs):
            newx=xs[i]-rwc[0]
            newy=ys[i]-rwc[1]
            EDs=math.sqrt(newx**2+newy**2)
            ED.append(EDs)
            i+=1
        if t == 'PRE' or t == 'PRO':
            nonrewardedvals.append(ED)
        if t == 'SAM' or t == 'CHO':
            rewardedvals.append(ED)

        vals=ED
        (mu, sigma) = stats.norm.fit(vals)
        n, bins, patches = plt.hist(vals, 50, density=True, facecolor='lightblue', align='left', edgecolor='royalblue', alpha=0.75)
#        y = stats.norm.pdf(bins, mu, sigma)
#        l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.4)
        locs, labels=plt.yticks()
        new=locs*100
        news=[]
        for i in new:
            news.append(round(i, 1))
            
        plt.yticks(ticks=locs, labels=news)
        meanse.append(mu)
        sdse.append(sigma)
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False
        plt.ylabel('Proportion of Events / %')
        plt.xlabel('Events distance from rewarded sandwell / Cm')
        plt.title(Animal +' '+ s+' ' + t + ' Event distibution ')
        plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202_Analysis\%s_%s_%s_Events_dist.png' %(Animal, s, t ))

In [ ]:
## overall population remapping rate - pre-training/pre-training ##

#A01/A02 PRE ALL CELLS#
App_rr=[]
i=0
while i<len(xydict['A01']['PRE']['pfc']):
    x=xydict['A01']['PRE']['pfc'][i][0]-xydict['A02']['PRE']['pfc'][i][0]
    y=xydict['A01']['PRE']['pfc'][i][1]-xydict['A02']['PRE']['pfc'][i][1]
    if x<0 and y>=0 or x>=0 and y<0:
        displacement=(-1)*math.sqrt(x**2+y**2)
    if x>=0 and y>=0 or x<0 and y<0:
        displacement=math.sqrt(x**2+y**2)
    App_rr.append(displacement) 
    i+=1
# P01 / P02 ALL CELLS #
Ppp_rr=[]
while i<len(xydict['P01']['PRE']['pfc']):
    x=xydict['P01']['PRE']['pfc'][i][0]-xydict['P02']['PRE']['pfc'][i][0]
    y=xydict['P01']['PRE']['pfc'][i][1]-xydict['P02']['PRE']['pfc'][i][1]
    if x<0 and y>=0 or x>=0 and y<0:
        displacement=(-1)*math.sqrt(x**2+y**2)
    if x>=0 and y>=0 or x<0 and y<0:
        displacement=math.sqrt(x**2+y**2)
    Ppp_rr.append(displacement) 
    i+=1
# A02 / P01 PRE - ALL CELLS #
APpp_rr=[]
i=0
while i<len(corr_vals):
    x=xydict['P01']['PRE']['pfc'][corr_vals[i][1]][0]-xydict['A02']['PRE']['pfc'][corr_vals[i][0]][0]
    y=xydict['P01']['PRE']['pfc'][corr_vals[i][1]][1]-xydict['A02']['PRE']['pfc'][corr_vals[i][0]][1]
    if x<0 and y>=0 or x>=0 and y<0:
        displacement=(-1)*math.sqrt(x**2+y**2)
    if x>=0 and y>=0 or x<0 and y<0:
        displacement=math.sqrt(x**2+y**2)
    APpp_rr.append(displacement) 
    i+=1
all_rr_pre=[]
for i in Ppp_rr:
    all_rr_pre.append(i)
for r in APpp_rr:
    all_rr_pre.append(r)
for i in App_rr:
    all_rr_pre.append(i)
    
(mu, sigma)=stats.norm.fit(all_rr_pre)
n, bins, patches = plt.hist(all_rr_pre, 40, density=True, facecolor='lightblue', alpha=0.75, align='left', edgecolor='royalblue' , linewidth=1.2)
y = stats.norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, linewidth=2, color='firebrick', alpha=0.7)
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.title('H2202 all cells remapping rate A01/A02/P01/P02 - PRE', pad=25)
plt.plot([], [], ' ', label="μ =-8.249035124114538 , σ =  39.12475708089716, 95th perc = 67.7844775296104")
plt.xlabel('Remapping distance / cm')
plt.ylabel('Proportion of cells / %')
locs, labels=plt.yticks()
new=locs*100
news=[]
for i in new:
    news.append(round(i, 1))
            
plt.yticks(ticks=locs, labels=news)
lgd=plt.legend(loc=1, prop={'size': 10}, bbox_to_anchor=(2.5, 1.05))
#plt.axvline(z, color='k', linestyle='dashed', linewidth=1)
plt.savefig(r'C:\Users\meera\Documents\Integrative Neuroscience Msc\H2202\Place cell figures\%s_A01_A02_P01_P02_remapping_rate_allcells_PREONLY.png' %(Animal), bbox_extra_artists=(lgd,), bbox_inches='tight')
print(mu, sigma)

In [ ]:
### global remapping distances stage/stage ###
A1PS=[]
i=0
while i<len(xydict['A01']['SAM']['pfc']):
    x=xydict['A01']['SAM']['pfc'][i][0]-xydict['A01']['PRE']['pfc'][i][0]
    y=xydict['A01']['SAM']['pfc'][i][1]-xydict['A01']['PRE']['pfc'][i][1]
    A1PS.append(math.sqrt(x**2+y**2))
    i+=1
A1SC=[]
i=0
while i<len(xydict['A01']['SAM']['pfc']):
    x=xydict['A01']['SAM']['pfc'][i][0]-xydict['A01']['CHO']['pfc'][i][0]
    y=xydict['A01']['SAM']['pfc'][i][1]-xydict['A01']['CHO']['pfc'][i][1]
    A1SC.append(math.sqrt(x**2+y**2))
    i+=1
A2PS=[]
i=0
while i<len(xydict['A02']['SAM']['pfc']):
    x=xydict['A02']['SAM']['pfc'][i][0]-xydict['A02']['PRE']['pfc'][i][0]
    y=xydict['A02']['SAM']['pfc'][i][1]-xydict['A02']['PRE']['pfc'][i][1]
    A2PS.append(math.sqrt(x**2+y**2))
    i+=1
A2SC=[]
i=0
while i<len(xydict['A02']['SAM']['pfc']):
    x=xydict['A02']['SAM']['pfc'][i][0]-xydict['A02']['CHO']['pfc'][i][0]
    y=xydict['A02']['SAM']['pfc'][i][1]-xydict['A02']['CHO']['pfc'][i][1]
    A2SC.append(math.sqrt(x**2+y**2))
    i+=1
P1PS=[]
i=0
while i<len(xydict['P01']['SAM']['pfc']):
    x=xydict['P01']['SAM']['pfc'][i][0]-xydict['P01']['PRE']['pfc'][i][0]
    y=xydict['P01']['SAM']['pfc'][i][1]-xydict['P01']['PRE']['pfc'][i][1]
    P1PS.append(math.sqrt(x**2+y**2))
    i+=1
P1SP=[]
i=0
while i<len(xydict['P01']['SAM']['pfc']):
    x=xydict['P01']['SAM']['pfc'][i][0]-xydict['P01']['PRO']['pfc'][i][0]
    y=xydict['P01']['SAM']['pfc'][i][1]-xydict['P01']['PRO']['pfc'][i][1]
    P1SP.append(math.sqrt(x**2+y**2))
    i+=1
P1PC=[]
i=0
while i<len(xydict['P01']['SAM']['pfc']):
    x=xydict['P01']['CHO']['pfc'][i][0]-xydict['P01']['PRO']['pfc'][i][0]
    y=xydict['P01']['CHO']['pfc'][i][1]-xydict['P01']['PRO']['pfc'][i][1]
    P1PC.append(math.sqrt(x**2+y**2))
    i+=1
P2PS=[]
i=0
while i<len(xydict['P02']['SAM']['pfc']):
    x=xydict['P02']['SAM']['pfc'][i][0]-xydict['P02']['PRE']['pfc'][i][0]
    y=xydict['P02']['SAM']['pfc'][i][1]-xydict['P02']['PRE']['pfc'][i][1]
    P2PS.append(math.sqrt(x**2+y**2))
    i+=1
P2SP=[]
i=0
while i<len(xydict['P02']['SAM']['pfc']):
    x=xydict['P02']['SAM']['pfc'][i][0]-xydict['P02']['PRO']['pfc'][i][0]
    y=xydict['P02']['SAM']['pfc'][i][1]-xydict['P02']['PRO']['pfc'][i][1]
    P2SP.append(math.sqrt(x**2+y**2))
    i+=1
P2PC=[]
i=0
while i<len(xydict['P02']['SAM']['pfc']):
    x=xydict['P02']['CHO']['pfc'][i][0]-xydict['P02']['PRO']['pfc'][i][0]
    y=xydict['P02']['CHO']['pfc'][i][1]-xydict['P02']['PRO']['pfc'][i][1]
    P2PC.append(math.sqrt(x**2+y**2))
    i+=1